<a href="https://colab.research.google.com/github/Stormlocc/Viajero_paralelo_AG/blob/main/4thre_Paralelizacion_IA_Proyecto_AG_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTELIGENCIA ARTIFICIAL

# Cargar puntos del mapa en un archivo puntos-dj38.tps

In [ ]:
%%writefile puntos.tps
42102.500000 11003.611100
42373.888900 11108.611100
42885.833300 11133.333300
42712.500000 11155.833300
42933.333300 11183.333300
42853.333300 11297.500000
42929.444400 11310.277800
42983.333300 11416.666700
43000.277800 11423.888900
42057.222200 11438.333300
43252.777800 11461.111100
43187.222200 11485.555600
42855.277800 11503.055600
42106.388900 11511.388900
42841.944400 11522.222200
43136.666700 11569.444400
43150.000000 11583.333300
43148.055600 11595.000000
43150.000000 11600.000000
42686.666700 11690.555600
41836.111100 11715.833300
42814.444400 11751.111100
42651.944400 11770.277800
42884.444400 11785.277800
42673.611100 11822.777800
42660.555600 11846.944400
43290.555600 11963.055600
43026.111100 11973.055600
42195.555600 12058.333300
42477.500000 12149.444400
43355.555600 12286.944400
42433.333300 12300.000000
43156.388900 12355.833300
43189.166700 12363.333300
42711.388900 12372.777800
43334.722200 12386.666700
42895.555600 12421.666700
42973.333300 12645.000000


Overwriting puntos.tps


# Diseñando el codigo

## Agregar las librerias

In [ ]:
# -*- coding: utf-8 -*-
# Creado por: Miriam del Carmen García Martínez gammc7@gmail.com
# Modificado para python3: Anthony Lopez Oquendo: github@Stormlocc
from math import sqrt, log
from itertools import permutations as permutaciones
from itertools import islice as rango
from random import randrange as aleatorio
from random import random
#Agregando otras librerias
import multiprocessing
import threading
import datetime

## Ciudades y distancias

In [ ]:
posicionCiudades = ['42102.500000 11003.611100\n', '42373.888900 11108.611100\n', '42885.833300 11133.333300\n', '42712.500000 11155.833300\n', '42933.333300 11183.333300\n', '42853.333300 11297.500000\n', '42929.444400 11310.277800\n', '42983.333300 11416.666700\n', '43000.277800 11423.888900\n', '42057.222200 11438.333300\n', '43252.777800 11461.111100\n', '43187.222200 11485.555600\n', '42855.277800 11503.055600\n', '42106.388900 11511.388900\n', '42841.944400 11522.222200\n', '43136.666700 11569.444400\n', '43150.000000 11583.333300\n', '43148.055600 11595.000000\n', '43150.000000 11600.000000\n', '42686.666700 11690.555600\n', '41836.111100 11715.833300\n', '42814.444400 11751.111100\n', '42651.944400 11770.277800\n', '42884.444400 11785.277800\n', '42673.611100 11822.777800\n', '42660.555600 11846.944400\n', '43290.555600 11963.055600\n', '43026.111100 11973.055600\n', '42195.555600 12058.333300\n', '42477.500000 12149.444400\n', '43355.555600 12286.944400\n', '42433.333300 12300.000000\n', '43156.388900 12355.833300\n', '43189.166700 12363.333300\n', '42711.388900 12372.777800\n', '43334.722200 12386.666700\n', '42895.555600 12421.666700\n', '42973.333300 12645.000000']


# Diccionario global donde se guardan las distancias de una ciudad a otra.
distancias = {}

# Funciones de algoritmos genetico

## Funciones adicionales a las clases

In [ ]:

# Metodo que calcula la desvion estándar.
def desvestandar(poblacion=[]):
    tam = len(poblacion)
    aptitudes = map(cromosoma.aptitud, poblacion)
    xprom = sum(aptitudes)/tam
    desvs = sqrt(sum(map(lambda x: (x-xprom)**2, aptitudes)) / (tam - 1))
    return desvs, xprom


# Invoca el metodo de mutacion para algunos individuos de la poblacion con
# cierta probabilidad.
def mutarpop(pop, pmuta):
    for p in pop:
        if pmuta > random():
            p.mutar()

# Cruza los individuos de la poblacion para obtener cromosomas nuevos que
# posteriormente se integrarán a la poblacion en la siguiente generacion.
def cruzarpob(pob, pcruza):
    padres = list(pob)
    hijos = list()

    if len(padres) % 2:
        if random() > 0.5:
            padres.pop(aleatorio(0, len(padres)))
        else:
            padres.append(padres[aleatorio(0, len(padres))])

    while padres:
        a = padres.pop(aleatorio(0, len(padres)))
        b = padres.pop(aleatorio(0, len(padres)))
        if pcruza > random():
            hijos.extend(a.cruzar(b))

    return hijos

# selecciona
def selecciona(poblacion, pobtam):
    p = list(poblacion)
    p.sort()
    return p[:pobtam]

# Metodo que calcula el factorial de un número.
def fact(n):
    return (1 if n < 2 else n * fact(n-1))


# Método que crea nuestras ciudades a partir de un archivo
def parserCiudades(archivo):
    fcd = open(archivo, "r")
    lineas = fcd.readlines()
    fcd.close()
    ciudades = []
    for i, linea in enumerate(lineas):
        nums = linea.split(" ")
        ciudades.append(ciudad(i, float(nums[0]), float(nums[1])))
    return ciudades

## Clase Ciudad = GEN

In [ ]:
# ciudad clase cuyas instancias representan las ciudades que forman parte del territorio
class ciudad:
    # constructor de la clase
    def __init__(self, id, x, y):
        self.__id = id
        self.__x = x
        self.__y = y

    # representación de una ciudad como cadena de caracteres
    def __str__(self):
        return "%d" % (self.__id)

    # devuelve el valor del id de la ciudad
    def getid(self):
        return self.__id

    # devuelve el valor de x de la ciudad
    def getx(self):
        return self.__x

    # devuelve el valor de y de la ciudad
    def gety(self):
        return self.__y

## Clase Cromosoma

In [ ]:
# cromosoma clase cuyas instancias representan posibles soluciones al problema de optimización
class cromosoma:
    # constructor de la clase
    def __init__(self, ciudades=[]):
        self.__ciudades = list(ciudades)

    # copia un cromosoma.
    def copy(self):
        return cromosoma(self.getciudades())

    # devuelve el valor de una ciudad
    def getciudades(self):
        return self.__ciudades

    # representación como cadena de caracteres del cromosoma
    def __str__(self):
        s = "["
        for ciudad in self.getciudades():
            s += " %s," % (str(ciudad))
        s += " %2s ]" % (self.getciudades()[0])
        return s

    # Método que calcula la distancia de una ciudad a otra.
    def distancia(self, cd1, cd2):
        a, b = (cd1, cd2) if cd1.getid() < cd2.getid() else (cd2, cd1)
        key = "%d-%d" % (a.getid(), b.getid())
        if distancias.__contains__(key):
            dist = distancias[key]
        else:
            x = b.getx() - a.getx()
            y = b.gety() - a.gety()
            dist = sqrt(x**2 + y**2)
            distancias[key] = dist
        return dist

    # Método que calcula la aptitud asociada a un cromosoma.
    # Método que calcula la distancia total de un recorrido de ciudades.
    def aptitud(self):
        apt = 0
        orig = self.getciudades()
        dest = list(orig)
        tmp = dest.pop(0)
        dest.append(tmp)
        for a, b in zip(orig, dest):
            apt += self.distancia(a, b)
        return apt

    # Método para encontrar la posición de una ciudad en la lista.
    def index(self, nodo):
        return self.___ciudades.index(nodo)

    # Método que implementa la cruza de dos cromosomas.
    # Cruzamiento por ciclos
    def cruzar(self, otro):
        p1, p2 = self.getciudades(), otro.getciudades()
        h1, h2 = list(p2), list(p1)
        idx = [0]
        i = 0
        cont = True
        while cont:
            ciudad = p2[i]
            i = p1.index(ciudad)
            idx.append(i)
            cont = not p1[0] is p2[i]

        for i in idx:
            h1[i], h2[i] = h2[i], h1[i]

        return cromosoma(h1), cromosoma(h2)

    # Método con el que un cromosoma muta.
    # Mutación por inversión.
    def mutar(self):
        lista = self.getciudades()
        mitad = len(lista)/2
        ini = aleatorio(0, mitad)
        fin = aleatorio(mitad, len(lista))
        a, b, c = lista[0:ini], lista[ini:fin], lista[fin:]
        b.reverse()
        del self.__ciudades
        self.__ciudades = a + b + c
        del a
        del b
        del c

# Clase Algoritmo Genetico Secuencial

In [ ]:
# Implementacion de un algoritmo genetico para resolver el problema del agente viajero.
class AlgoritmoGenetico:
    def __init__(self,ciudades, pobtam, pcruza, pmuta, iteraciones):
        # -- Poblacion inicial
        self.poblacion = self.generapob(ciudades, pobtam)
        print("\t POBLACION INICIAL")
        print(self.MostrarPoblacion(self.poblacion))
        # -- Aplicar funcion de aptitud para encontrar mejores
        self.mejores = list()
        self.poblacion.sort(key= lambda cromosoma : cromosoma.aptitud())
        mejor = self.poblacion[0]
        self.mejores.append(mejor.copy())
        selec = self.seleccionTorneo(self.poblacion, pobtam)

        # X/4       2x/4      3x/4      4x/4
        # Iniciar el proceso
        p1 = threading.Thread(name = "hilo 1", target = self.iniciar , args = (iteraciones//4 , selec[:pobtam//4] ,pcruza,pobtam//4,pmuta) )
        p2 = threading.Thread(name = "hilo 2", target = self.iniciar , args = (iteraciones//4, selec[pobtam//4:2*pobtam//8] ,pcruza,pobtam//4,pmuta) )
        p3 = threading.Thread(name = "hilo 3", target = self.iniciar , args = (iteraciones//4, selec[2*pobtam//4:3*pobtam//4] ,pcruza,pobtam//4,pmuta) )
        p4 = threading.Thread(name = "hilo 4", target = self.iniciar , args = (iteraciones//4, selec[3*pobtam//4:pobtam] ,pcruza,pobtam//4,pmuta) )


        p1.start()
        p2.start()
        p3.start()
        p4.start()

        p1.join()
        p2.join()
        p3.join()
        p4.join()



    def iniciar(self,iteraciones,selec,pcruza,pobtam,pmuta):
        continua = True
        for i in range(1,iteraciones):
          if continua:
            print(i,") comenzando iteracion")
            #print(self.MostrarPoblacion(selec))
            # -- Cruce
            hijos = cruzarpob(selec, pcruza)
            selec.extend(hijos)
            # -- Mutacion
            mutarpop(selec, pmuta)
            # -- Nueva poblacion en mejores
            selec = self.seleccionTorneo(selec, pobtam)
            selec.sort(key= lambda cromosoma : cromosoma.aptitud())
            self.mejores.append(selec[0].copy())
            if(len(self.mejores) > 1):
                aux = list(self.mejores)
                aux.sort(key= lambda cromosoma : cromosoma.aptitud())
                n = aux.count(aux[0])
                if(len(self.mejores) >= 10 and n > len(aux)/2):
                    continua = False


    # -- Genera la poblacion inicial.
    def generapob(self,ciudades, pob):
        s = list()
        perms = permutaciones(ciudades)
        fin = aleatorio(pob, fact(len(ciudades)))
        ini = fin - pob
        Sigma = list(rango(perms, pob))
        for sigma in Sigma:
            k = list(sigma)
            s.append(cromosoma(k))
        for k in range(2):
            mutarpop(s, 1.0)
        return s

    # -- Funcion de seleccion (por torneo).
    def seleccionTorneo(self, poblacion, tampob):
        selec = list()
        for k in [1, 2]:
            participantes = list(poblacion)
            if len(participantes) % 2 == 1:
                if k == 1:
                    participantes.pop(aleatorio(0, len(participantes)))
                else:
                    participantes.append(
                        participantes[aleatorio(0, len(participantes))])
            while participantes:
                a = participantes.pop(aleatorio(0, len(participantes)))
                b = participantes.pop(aleatorio(0, len(participantes)))
                ganador = a if a.aptitud() < b.aptitud() else b
                selec.append(ganador)
        selec.sort(key= lambda cromosoma : cromosoma.aptitud())
        return selec[:tampob]


    # Genera una representacion como cadena de caracteres de un conjunto de
    # soluciones.
    def MostrarPoblacion(self,pob):
        n = int(log(len(pob), 10) + 1)
        s = " | %" + str(n) + "d | %s | %f\n"
        #edge = " +" + (n + 2) * "-" + "+" + (len(str(pob[0])) + 2) * "-" + "+ APTITUD \n"
        edge = " +--i-+" + (len(str(pob[0])) + 2) * "-" + "+ --APTITUD--+ \n"
        strpob = edge
        for i, p in enumerate(pob):
            strpob += s % (i+1, str(p), p.aptitud())

        return strpob + edge

    def MostrarMejor(self):
        self.mejores.sort(key= lambda cromosoma : cromosoma.aptitud())
        print("\n\t SOLUCION DEL PROBLEMA")
        optimo = self.mejores[0]
        print("Mejor solucion encontrada:")
        print(optimo)
        print("Con un costo de "+ str(optimo.aptitud()) + " unidades.")

## Programa principal

In [ ]:
def main():
    ciudades = parserCiudades("puntos.tps")
    pobtam = 40
    pcruza = 0.9
    pmuta = 0.5
    iteraciones = 2600
    ti = datetime.datetime.now()
    optimo = AlgoritmoGenetico(ciudades, pobtam, pcruza, pmuta, iteraciones)
    tf = datetime.datetime.now()
    optimo.MostrarMejor()
    print("Finalizo")
    print("TIEMPO: ", tf-ti)

if __name__ == "__main__":
    main()
#Verifiar si ptam funciona hasta 40 de ahi ya no importa


Exception in thread hilo 2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-e8a14b2102b0>", line 49, in iniciar
IndexError: list index out of range


	 POBLACION INICIAL
 +--i-+------------------------------------------------------------------------------------------------------------------------------------------------------+ --APTITUD--+ 
 |  1 | [ 26, 25, 24, 23, 22, 21, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 7, 6, 5, 4, 3, 2, 1, 0, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 26 ] | 17631.900121
 |  2 | [ 0, 1, 2, 3, 4, 5, 6, 7, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 33, 32, 31, 30, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 34, 35, 37, 36,  0 ] | 18639.772756
 |  3 | [ 0, 1, 2, 3, 4, 5, 6, 7, 34, 33, 32, 31, 30, 29, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 11, 10, 9, 8, 36, 35, 37,  0 ] | 19927.319778
 |  4 | [ 0, 1, 2, 3, 4, 5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 8, 7, 6, 17, 16, 15, 14, 13, 12, 11, 10, 9, 31, 32, 33, 34, 36, 37, 35,  0 ] | 17372.139772
 |  5 | [ 0, 1, 2, 3, 4, 5, 6, 28, 29, 30, 31, 32, 33, 34, 37, 12, 11, 10, 9, 8, 7, 27, 26, 25, 24, 23, 22, 21, 20, 19,